# Prerequisites
Install the `OpenPAI` sdk from `github` and specify your cluster information in `~/openpai.json` (or somewhere you like). (Refer to for [0-install-sdk-specify-openpai-cluster.ipynb]() more details.)

<font color=red>Please make sure you have set default values for ***cluster-alias***, ***image*** and ***workspace***.</font>

In [ ]:
! opai cluster list
! opai default list

# Prepare your script locally
We prepare an example to show how reuse the same code in local environment and in `OpenPAI` node.

In [ ]:
%%writefile test.py

import os
import argparse
from openpaisdk.core import in_job_container

def say_hi(name):
    if in_job_container():
        print('Hello, {}! I am in a container: {}'.format(name, os.environ['PAI_CONTAINER_ID']))
    else:
        print('Hello, {}! I am from local script: {}'.format(name, __file__))

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('name', help='your name')
    args = parser.parse_args()
    say_hi(args.name)

In [ ]:
! python test.py Alice

# Submit to `OpenPAI` step by step
Now we will submit above command to `OpenPAI` through the `opai job` tool. 

In [ ]:
! opai job -h

Since `OpenPAI` doesnot allow duplicated job name, you must define a unique name with `-j <job-name>`. In below commands, we show how to create a job, add packages to install, add task role commands, and the submit it. 

To simplify user input, the `create` command would set current job as default (by setting default `job-name` to current), and the `submit` command would release the default (by removing `job-name` in default). To disable this, use `--dont-set-as-default` in `create`. 

In [ ]:
from uuid import uuid4 as randstr
job_name = 'test_new_cli_' + randstr().hex
print("newly generated job name", job_name)

! opai job create -j {job_name}

# pip install simplejson
! opai job create -u -p simplejson

! opai job task -t main python test.py Alice

! opai job submit

By clicking the printed job link, you may find below line in the `STDOUT`

```
Hello, Alice! I am in a container: container_e26_1556418566879_0058_01_000002
```
<font color=red>_The auto fetch of command output is under development_</font>

## submit by fastpass (Inprogress)

In [ ]:
! opai submit -i jupyter/minimal-notebook python test.py Alice